In [24]:
import os 
from google.cloud import bigquery
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'credentials/service_account.json'
import pandas as pd


In [39]:
from sklearn.datasets import make_classification
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import math
from sklearn.preprocessing import StandardScaler
import numpy as np



In [110]:
query = open('queries\dau_conversion').read()
df = pd.read_gbq(query, project_id = 'capim-404203', dialect= 'standard')
df['TRIAL_DURATION'] = df['TRIAL_DURATION'].astype('int')
df['CLINIC_ID'] = df['CLINIC_ID'].astype('int')
df['IS_CONVERTED'] = df['IS_CONVERTED'].replace({True: 1, False: 0})
df['IS_CONVERTED'] = df['IS_CONVERTED'].astype('int')
df['ACTIVE_PATIENT_DAYS'] = df['ACTIVE_PATIENT_DAYS'].astype('float')
df['ACTIVE_FINANCE_DAYS'] = df['ACTIVE_FINANCE_DAYS'].astype('float')
df['ACTIVE_SETUP_DAYS'] = df['ACTIVE_SETUP_DAYS'].astype('float')
df['ACTIVE_SCHEDULE_DAYS'] = df['ACTIVE_SCHEDULE_DAYS'].astype('float')
df['ACTIVE_USER_DAYS'] = df['ACTIVE_USER_DAYS'].astype('float') * 100
df['ACTIVE_MODULES_USED'] = (df['ACTIVE_PATIENT_DAYS'] > 0).astype('int') + (df['ACTIVE_FINANCE_DAYS'] > 0).astype('int') + (df['ACTIVE_SETUP_DAYS'] > 0).astype('int') + (df['ACTIVE_SCHEDULE_DAYS'] > 0).astype('int')



In [123]:
Y = df['IS_CONVERTED']
X = df[['TRIAL_DURATION','ACTIVE_FINANCE_DAYS', 'ACTIVE_PATIENT_DAYS', 'ACTIVE_SETUP_DAYS', 'ACTIVE_SCHEDULE_DAYS', 'ACTIVE_USER_DAYS']]
#X =  df[['ACTIVE_USER_DAYS_RAW']]

In [124]:
scaler = StandardScaler()
scaler.fit(X)
X_transformed = scaler.transform(X)

In [130]:
x_train, x_test, y_train, y_test = train_test_split(X_transformed, Y, random_state=35)

# Create a Logistic Regression Object, perform Logistic Regression
log_reg = LogisticRegression(C=.005)
log_reg.fit(x_train, y_train)

log_reg.get_params()
# Show to Coeficient and Intercept
print(log_reg.coef_)
print(log_reg.intercept_)

y_pred = log_reg.predict(x_test)


#Show the Confusion Matrix
confusion_matrix(y_test, y_pred)

[[0.02423318 0.18461357 0.10405509 0.13448864 0.11949563 0.24422984]]
[-1.62285687]


array([[406,   4],
       [ 91,  14]], dtype=int64)

In [132]:
log_reg.score(x_test,y_test)

0.8155339805825242

In [44]:
new_x = pd.DataFrame(list(np.arange(0,1.1,.1)),columns=['ACTIVE_USER_DAYS'])
probability_of_conversion = log_reg.predict_proba(new_x)[:,-1]
nex_table = new_x.copy()
nex_table['probability'] = probability_of_conversion

nex_table.to_csv('probs_of_conversion.csv')
nex_table


In [36]:
list(np.arange(0,1.1,.1))

[0.0,
 0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6000000000000001,
 0.7000000000000001,
 0.8,
 0.9,
 1.0]

In [54]:
Y_prediction = log_reg.predict(X)

Y_chance = log_reg.predict_proba(X)[:,-1]

In [56]:
df['PREDICTION_IS_CONVERTED'] = Y_prediction
df['PREDICTION_CONVERTED_CHANCE'] = Y_chance

In [58]:
df.to_gbq('capim.prediction_converted', if_exists='append')

In [196]:
import numpy as np